![Texto Alternativo](banco_de_dados_projeto.png)

Dados de vendas de 2016~2018 da Olist ecommerce

https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce#olist_products_dataset.csv

https://www.kaggle.com/datasets/thiagobodruk/brazil-geojson

https://github.com/filipegmedeiros/brazilian_choropleth_map/tree/main/geojson

In [1]:
import pandas as pd
import plotly.express as px
import json
import plotly.io as pio
pio.renderers.default = 'jupyterlab'

In [2]:
# Carga inicial dos dados
df_order_payments = pd.read_csv("dados/olist_order_payments_dataset.csv")
df_order_dataset = pd.read_csv("dados/olist_orders_dataset.csv")
df_order_customer = pd.read_csv("dados/olist_customers_dataset.csv")
df_order_itens = pd.read_csv("dados/olist_order_items_dataset.csv")
df_order_products = pd.read_csv("dados/olist_products_dataset.csv")
df_order_sellers = pd.read_csv("dados/olist_sellers_dataset.csv")
df_order_geolocation = pd.read_csv("dados/olist_geolocation_dataset.csv")

# Geojson
geojson = json.load(open("geojson/brasil_estados.json"))

In [3]:
# Merge dos dados
df_order_geolocation.rename(columns={'geolocation_zip_code_prefix' : 'zip_code_prefix'}, inplace=True)
df_unido = pd.merge(df_order_payments, df_order_dataset, how="inner", on="order_id")
df_unido = pd.merge(df_unido, df_order_itens, how="inner", on="order_id")
df_unido = pd.merge(df_unido, df_order_customer, how="inner", on="customer_id")
df_unido = pd.merge(df_unido, df_order_products, how="inner", on="product_id")
df_unido = pd.merge(df_unido, df_order_sellers, how="inner", on="seller_id")
df_unido.rename(columns={'customer_zip_code_prefix' : 'zip_code_prefix'}, inplace=True)

In [120]:
df_unido_amostra = df_unido.sample(frac=0.2, random_state=42)
df_unido_amostra.dropna(inplace=True)
#df_unido_amostra.to_csv("dados/amostra.csv")

In [ ]:
maiusc = lambda x: x.title()
df_unido_amostra['seller_city'] = df_unido_amostra['seller_city'].apply(maiusc)
df_unido_amostra['seller_city'].head()

In [ ]:
tipo_pagamento = df_unido_amostra.groupby(by='payment_type').size().reset_index().sort_values(by=0, ascending=False)
tipo_pagamento = tipo_pagamento.rename(columns={'payment_type': 'forma de pagamento', 0: 'quantidade',})

#mapeamento = {'credit_card': 'crédito', 'debit_card': 'débito', 'voucher':'voucher', 'boleto':'boleto'}
#tipo_pagamento['forma de pagamento'] = tipo_pagamento['forma de pagamento'].map(mapeamento)
tipo_pagamento.head()

In [ ]:
df_unido.columns

In [ ]:
print(df_unido_amostra['payment_installments'].max())
print(df_unido_amostra['payment_installments'].min())
print(df_unido_amostra['payment_installments'].mean().round())

In [ ]:
df_unido_amostra.groupby('product_category_name').size().reset_index().sort_values(by=0, ascending=False).head(10)

In [ ]:
compra_por_estado = df_unido_amostra.groupby(by='customer_state').size().sort_values(ascending=False).reset_index()

In [ ]:
# location tem que fazer correlação com a coluna ID do geojson
fig = px.choropleth(compra_por_estado, geojson=geojson, locations='customer_state', color=0, scope="south america")
fig.update_geos(fitbounds = "locations", visible = False)
fig.update_layout(width=800, height=800,dragmode=False,paper_bgcolor="white")
fig.show()

In [ ]:
compra_por_cidade = df_unido_amostra.groupby(by='customer_city').size().sort_values(ascending=False).reset_index()

In [132]:
df_unido_amostra.columns

Index(['order_id', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value', 'customer_id', 'order_status',
       'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'order_item_id', 'product_id',
       'seller_id', 'shipping_limit_date', 'price', 'freight_value',
       'customer_unique_id', 'zip_code_prefix', 'customer_city',
       'customer_state', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'seller_zip_code_prefix', 'seller_city', 'seller_state',
       'tempo de entrega', 'tempo estimado'],
      dtype='object')

In [134]:
df_unido_amostra[['order_approved_at', 'order_delivered_carrier_date']].head()

,order_approved_at,order_delivered_carrier_date
49672,2018-07-01,2018-07-02 16:24:00
23948,2017-12-06,2017-12-11 16:13:09
105553,2018-04-17,2018-04-17 20:24:35
68942,2017-05-17,2017-05-18 10:35:23
80847,2017-08-16,2017-08-18 19:22:18


In [131]:
# Thaysa
# Tempo de entrega
# Talvez tenha que usar a order carrier date...
df_unido_amostra['order_approved_at'] = pd.to_datetime(df_unido_amostra['order_approved_at']).dt.date
df_unido_amostra['order_delivered_customer_date'] = pd.to_datetime(df_unido_amostra['order_delivered_customer_date']).dt.date
df_unido_amostra['order_estimated_delivery_date'] = pd.to_datetime(df_unido_amostra['order_estimated_delivery_date']).dt.date

df_unido_amostra['tempo de entrega'] = (pd.to_timedelta(df_unido_amostra['order_delivered_customer_date'] - 
                                                       df_unido_amostra['order_approved_at']).dt.days).astype(int)

df_unido_amostra['tempo estimado'] = (pd.to_timedelta(df_unido_amostra['order_estimated_delivery_date'] - 
                                                     df_unido_amostra['order_approved_at']).dt.days).astype(int)


tempo_de_entrega = df_unido_amostra[['tempo de entrega', 'tempo estimado']]
tempo_de_entrega[tempo_de_entrega['tempo de entrega'] > 0]

,tempo de entrega,tempo estimado
49672,25,30
23948,22,29
105553,9,27
68942,2,13
80847,13,28
...,...,...
15085,16,27
16613,15,29
73988,19,12
38142,27,28


In [ ]:
# Luana
# Estados com mais clientes
clientes_por_estado = df_unido_amostra.groupby('customer_state')['customer_id'].nunique().sort_values(ascending=False).reset_index()

# Valor médio do frete por estado
media_frete_por_estado = df_unido_amostra.groupby('customer_state')['freight_value'].mean().sort_values(ascending=False).reset_index()

# Estados com mais vendedores
vendedores_por_estado = df_unido_amostra.groupby('seller_state')['seller_id'].nunique().sort_values(ascending=False).reset_index()

# Unindo os dataframes
merged_df = clientes_por_estado.merge(media_frete_por_estado, on='customer_state')
merged_df = merged_df.merge(vendedores_por_estado, how='outer', left_on='customer_state', right_on='seller_state')
merged_df.rename(columns={'customer_id': 'num_clientes', 'freight_value': 'valor_medio_frete', 'seller_id': 'num_vendedores'}, inplace=True)
merged_df = merged_df.sort_values(by='num_clientes', ascending=False)